# Sentiment analysis using Python

Part-of-speech tagging is the process of converting a sentence, in the form of a list of words, into a list of tuples, where each tuple is of the form (word, tag). The tag is a part-of-speech tag, and signifies whether the word is a noun, adjective, verb, and so on.

The main goal of this notebook is to identify if reviews of movies are positives or negatives.

In [1]:
import nltk

Most of the taggers are trainable. They use a list of tagged sentences as their training data. With these training sentences, the tagger generates an internal model that will tell it how to tag a word. Other taggers use external data sources or match word patterns to choose a tag for a word.

Here we will use UnigramTagger by giving it a list of tagged sentences at initialization.

In [2]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/nathanamar/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
train_sents = treebank.tagged_sents()
tagger = UnigramTagger(train_sents)
treebank.sents()[0]

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 ',',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.']

In [4]:
tagger.tag(treebank.sents()[0])

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

We see the first sentence as a list of words, and can see how it is transformed by the tag() function into a list of tagged tokens.

To identify positivity or negativity of the reviews we will use SentiWordNet, a lexical resource for opinion mining. SentiWordNet assigns to each synset of WordNet three sentiment scores: positivity, negativity, objectivity.

In [5]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/nathanamar/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [6]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('good', 'a'))[0].pos_score()

0.75

In [7]:
list(swn.senti_synsets('good', 'a'))[0].neg_score()

0.0

In [8]:
list(swn.senti_synsets('good', 'a'))[0].obj_score()

0.25

## Processing the data

We first have to process the reviews.

This is a function that helps us replace words matching regular expressions.

In [9]:
import re

replacement_patterns = [
    (r'’', '\''),
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would'),
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns): 
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s) 
        return s
    

replacer=RegexpReplacer()
replacer.replace("Don't hesistate to ask questions")

'Do not hesistate to ask questions'

In [10]:
open_file = open('/Users/nathanamar/Desktop/ESILV/A5/Advanced Machine Learning for Big Data Text Processing/WP2/data/review_polarity/txt_sentoken/pos/cv000_29590.txt', 'r', encoding='utf-8')
file_to_string = open_file.read()
type(file_to_string)

str

In [11]:
text_replaced = replacer.replace(file_to_string)
print(file_to_string[50:100])
print(text_replaced[50:100])
print(type(text_replaced))

success , whether they're about superheroes ( batm
success , whether they are about superheroes ( bat
<class 'str'>


After replacing the words by regular expressions, we will tokenize the reviews in a list of sentences, and then in a list of words.

In [12]:
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(text_replaced)
len(sentences)#27
sentences[0]

'films adapted from comic books have had plenty of success , whether they are about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there is never really been a comic book like from hell before .'

In [13]:
sentences[1]

"for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen ."

In [14]:
from nltk.tokenize import RegexpTokenizer
tokenizer=RegexpTokenizer("[\w]+")

for i in range(len(sentences)):
    sentences[i] = tokenizer.tokenize(sentences[i])
sentences[0]

['films',
 'adapted',
 'from',
 'comic',
 'books',
 'have',
 'had',
 'plenty',
 'of',
 'success',
 'whether',
 'they',
 'are',
 'about',
 'superheroes',
 'batman',
 'superman',
 'spawn',
 'or',
 'geared',
 'toward',
 'kids',
 'casper',
 'or',
 'the',
 'arthouse',
 'crowd',
 'ghost',
 'world',
 'but',
 'there',
 'is',
 'never',
 'really',
 'been',
 'a',
 'comic',
 'book',
 'like',
 'from',
 'hell',
 'before']

We will now use the tagger trained earlier using the UnigramTagger to tag each word of all sentences.

In [15]:
tagged_sent =[]
for sentence in sentences:
    tagged_sent.append(tagger.tag(sentence))
tagged_sent[0]

[('films', 'NNS'),
 ('adapted', 'VBD'),
 ('from', 'IN'),
 ('comic', None),
 ('books', 'NNS'),
 ('have', 'VBP'),
 ('had', 'VBD'),
 ('plenty', 'NN'),
 ('of', 'IN'),
 ('success', 'NN'),
 ('whether', 'IN'),
 ('they', 'PRP'),
 ('are', 'VBP'),
 ('about', 'IN'),
 ('superheroes', None),
 ('batman', None),
 ('superman', None),
 ('spawn', None),
 ('or', 'CC'),
 ('geared', None),
 ('toward', 'IN'),
 ('kids', 'NNS'),
 ('casper', None),
 ('or', 'CC'),
 ('the', 'DT'),
 ('arthouse', None),
 ('crowd', 'NN'),
 ('ghost', None),
 ('world', 'NN'),
 ('but', 'CC'),
 ('there', 'EX'),
 ('is', 'VBZ'),
 ('never', 'RB'),
 ('really', 'RB'),
 ('been', 'VBN'),
 ('a', 'DT'),
 ('comic', None),
 ('book', 'NN'),
 ('like', 'IN'),
 ('from', 'IN'),
 ('hell', None),
 ('before', 'IN')]

The tags used by *SentiWordNet* are different than the tags of the UnigramTagger. For example, an adjectif is tagged as **_'JJ'_**  in our tagger and it is tagged as **_'a'_** in *SentiWordNet*.

The function below allows us to get the 3 scores (positive, negative, objective) of a word by using a tuple *(word, tag)* as an argument.

In [16]:
#Recall
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('kill', 'v'))[0].neg_score()

0.5

In [17]:
#Recall
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('beautiful', 'a'))[0].pos_score()

0.75

In [18]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('good', 'a'))[0].pos_score()

0.75

In [19]:
list(swn.senti_synsets('never', 'r'))[0].neg_score() #R MEANS adverb = RB

0.625

In [20]:
def word_scores(wordntag):
    result = []
    word, tag = wordntag
    if(tag == 'JJ'):
        if( len(list(swn.senti_synsets(word, 'a'))) != 0 ):
            result.extend([list(swn.senti_synsets(word, 'a'))[0].pos_score(), list(swn.senti_synsets(word, 'a'))[0].neg_score(), list(swn.senti_synsets(word, 'a'))[0].obj_score()])
        else:
            result = [0.0, 0.0, 0.0]
    elif (tag == 'NNS' or tag == 'NN'):
        if( len(list(swn.senti_synsets(word, 'n'))) != 0 ):
            result.extend([list(swn.senti_synsets(word, 'n'))[0].pos_score(), list(swn.senti_synsets(word, 'n'))[0].neg_score(), list(swn.senti_synsets(word, 'n'))[0].obj_score()])
        else:
            result = [0.0, 0.0, 0.0]
    elif(tag == 'RB'):
        if( len(list(swn.senti_synsets(word, 'r'))) != 0 ):
            result.extend([list(swn.senti_synsets(word, 'r'))[0].pos_score(), list(swn.senti_synsets(word, 'r'))[0].neg_score(), list(swn.senti_synsets(word, 'r'))[0].obj_score()])
        else:
            result = [0.0, 0.0, 0.0]
    elif(tag == 'VBD' or tag == 'VBP' or tag == 'VBN' or tag == 'VBZ'):
        if( len(list(swn.senti_synsets(word, 'v'))) != 0 ):
            result.extend([list(swn.senti_synsets(word, 'v'))[0].pos_score(), list(swn.senti_synsets(word, 'v'))[0].neg_score(), list(swn.senti_synsets(word, 'v'))[0].obj_score()])
    else:
        result = [0.0, 0.0, 0.0]
    return result;

print(word_scores(tagged_sent[0][0]))
print(word_scores(tagged_sent[0][9]))#Sucess
print(word_scores(tagged_sent[0][32])) # with the word never
print(word_scores(tagged_sent[6][39]))#objectif 1 for crime
print(word_scores(tagged_sent[14][3])) # 0.
print(word_scores(tagged_sent[0][12]))# 0.25 positif 

[0.0, 0.0, 1.0]
[0.125, 0.0, 0.875]
[0.0, 0.625, 0.375]
[0.0, 0.0, 1.0]
[0.75, 0.0, 0.25]
[0.25, 0.125, 0.625]


In [21]:
tagged_sent[14][3]

('good', 'JJ')

In [22]:
tagged_sent[0][12]

('are', 'VBP')

In [23]:
#TEST WITH THE VBP ARE
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('are', 'v'))[0].pos_score()

0.25

In [24]:
tagged_sent[0][9]

('success', 'NN')

In [25]:
tagged_sent[0][1]

('adapted', 'VBD')

In [26]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('adapted', 'v'))[0].pos_score()

0.25

We apply that function for every word of each sentence to get a list of scores.

In [27]:
tagged_sent

[[('films', 'NNS'),
  ('adapted', 'VBD'),
  ('from', 'IN'),
  ('comic', None),
  ('books', 'NNS'),
  ('have', 'VBP'),
  ('had', 'VBD'),
  ('plenty', 'NN'),
  ('of', 'IN'),
  ('success', 'NN'),
  ('whether', 'IN'),
  ('they', 'PRP'),
  ('are', 'VBP'),
  ('about', 'IN'),
  ('superheroes', None),
  ('batman', None),
  ('superman', None),
  ('spawn', None),
  ('or', 'CC'),
  ('geared', None),
  ('toward', 'IN'),
  ('kids', 'NNS'),
  ('casper', None),
  ('or', 'CC'),
  ('the', 'DT'),
  ('arthouse', None),
  ('crowd', 'NN'),
  ('ghost', None),
  ('world', 'NN'),
  ('but', 'CC'),
  ('there', 'EX'),
  ('is', 'VBZ'),
  ('never', 'RB'),
  ('really', 'RB'),
  ('been', 'VBN'),
  ('a', 'DT'),
  ('comic', None),
  ('book', 'NN'),
  ('like', 'IN'),
  ('from', 'IN'),
  ('hell', None),
  ('before', 'IN')],
 [('for', 'IN'),
  ('starters', 'NNS'),
  ('it', 'PRP'),
  ('was', 'VBD'),
  ('created', 'VBN'),
  ('by', 'IN'),
  ('alan', None),
  ('moore', None),
  ('and', 'CC'),
  ('eddie', None),
  ('campbell'

In [28]:
scores = []
for sentence in tagged_sent:
    list_scores = []
    for word in sentence:
        list_scores.append(word_scores(word))
    scores.append(list_scores)
print(scores[0])

[[0.0, 0.0, 1.0], [0.25, 0.0, 0.75], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.25, 0.0, 0.75], [0.25, 0.0, 0.75], [0.0, 0.375, 0.625], [0.0, 0.0, 0.0], [0.125, 0.0, 0.875], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.25, 0.125, 0.625], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.25, 0.125, 0.625], [0.0, 0.625, 0.375], [0.625, 0.0, 0.375], [0.25, 0.125, 0.625], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


## Scoring

The question is now to determine how will we decide if a review is positive or negative.

**First approach : we decide based on majority**

If the positive score is bigger than the negative one, then it is a positive review. Else it's a negative one.

Let's first sum the positive and negative scores for each sentence.

In [29]:
sum_score = []
for list_score in scores:
    pos, neg, obj = 0.0, 0.0, 0.0
    for score in list_score:
        pos += score[0]
        neg += score[1]
        obj += score[2]
    if(len(list_score) != 0):
        sum_score.append([pos/len(list_score), neg/len(list_score), obj/len(list_score)])
    else:
        sum_score.append([0.0, 0.0, 0.0])
sum_score[0]  

[0.05357142857142857, 0.03273809523809524, 0.29464285714285715]

Then we sum the scores of all sentences to get the global score of the review.

In [30]:
pos, neg, obj = 0.0, 0.0, 0.0
for score in sum_score:
    pos += score[0]
    neg += score[1]
    obj += score[2]
pos /= len(sum_score)
neg /= len(sum_score)
obj /= len(sum_score)
print([pos, neg, obj])

[0.0361269223282653, 0.03247102490788594, 0.2605192039899847]


Creating a function that does the whole process for a text put in argument.

In [31]:
def sumScores(file):
    open_file = open(file, 'r', encoding='utf-8')
    file_to_string = open_file.read()
    
    text_replaced = replacer.replace(file_to_string)
    
    tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(text_replaced)
    
    from nltk.tokenize import RegexpTokenizer
    tokenizer=RegexpTokenizer("[\w]+")

    for i in range(len(sentences)):
        sentences[i] = tokenizer.tokenize(sentences[i])
        
    tagged_sent = []
    for sentence in sentences:
        tagged_sent.append(tagger.tag(sentence))
        
    scores = []
    for sentence in tagged_sent:
        list_scores = []
        for word in sentence:
            list_scores.append(word_scores(word))
        scores.append(list_scores)
        
    sum_score = []
    for list_score in scores:
        pos, neg, obj = 0.0, 0.0, 0.0
        for score in list_score:
            pos += score[0]
            neg += score[1]
            obj += score[2]
        if(len(list_score) != 0):
            sum_score.append([pos/len(list_score), neg/len(list_score), obj/len(list_score)])
        else:
            sum_score.append([0.0, 0.0, 0.0])
        
    pos, neg, obj = 0.0, 0.0, 0.0
    for score in sum_score:
        pos += score[0]
        neg += score[1]
        obj += score[2]
    if(len(sum_score) != 0):
        pos /= len(sum_score)
        neg /= len(sum_score)
        obj /= len(sum_score)
    return([pos, neg, obj])
    
sumScores('data/review_polarity/txt_sentoken/pos/cv000_29590.txt')

[0.0361269223282653, 0.03247102490788594, 0.2605192039899847]

Let's try that scoring technique on all positive reviews and see how it performs.

__First decision :__ We choose the highest value (between neg and pos) to determine if a review is positive or negative.

In [32]:
import os
pos_reviews = os.listdir('data/review_polarity/txt_sentoken/pos')
neg_reviews = os.listdir('data/review_polarity/txt_sentoken/pos')

In [33]:
pos, neg = 0, 0
for review in pos_reviews:
    if(review[0] != 'c'):
        continue
    score = sumScores('data/review_polarity/txt_sentoken/pos/'+review)
    if(score[0] > score[1]):
        pos += 1
    else:
        neg +=1
print(pos, neg)

846 154
